In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [2]:
fisheye_params = {
    "width": 2592,
    "height": 1944,
    "cx": 1296.0,
    "cy": 972.0,
    "radius": 952.56,
    "theta_max_rad": 1.5708
}

cam_settings = [
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": -18.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 54.0, "yaw": 48.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 28.0, "yaw": 150.0},
    {"cx": 1296.0, "cy": 972.0, "fov": 90.0, "pitch": 58.0, "yaw": 294.0},
]

views_path = [
    "captures_yaw_pitch/capture1/view1_1.jpg",
    "captures_yaw_pitch/capture1/view2_1.jpg",
    "captures_yaw_pitch/capture1/view3_1.jpg",
    "captures_yaw_pitch/capture1/view4_1.jpg"
]
fisheye_path = "captures_yaw_pitch/capture1/fisheye_1.jpg"
BEV = cv2.imread("ohlf.png")   
h = np.load("Homography for 229_camera_.npy")

def load_images(views, fisheye_path=None):
    image_bgr = []
    for p in views:
        b = cv2.imread(p)
        if b is None:
            raise FileNotFoundError(f"Missing {p}")
        image_bgr.append(b)
    fisheye_img = None
    if fisheye_path:
        fisheye_img = cv2.imread(fisheye_path)
        
    return image_bgr, fisheye_img
views, fisheye_img = load_images(views_path, fisheye_path)
rect_h, rect_w = views[0].shape[:2]

def compute_basis(yaw_deg, pitch_deg):
    yaw_deg_corrected = yaw_deg - 90.0
    yaw, pitch = np.deg2rad([yaw_deg_corrected, pitch_deg])

    Rz = np.array([
        [ np.cos(-yaw), -np.sin(-yaw), 0],
        [ np.sin(-yaw),  np.cos(-yaw), 0],
        [ 0,             0,            1]
    ])
    Rx = np.array([
        [ 1, 0,           0          ],
        [ 0, np.cos(pitch), -np.sin(pitch)],
        [ 0, np.sin(pitch),  np.cos(pitch)]
    ])
    R_ = Rx @ Rz
    R = np.diag([1, -1, 1]) @ R_
    return R

def rect_to_fisheye_point(u, v, view_params, fisheye_params):
    Hr, Wr = rect_h, rect_w
    cx_r, cy_r = Wr / 2, Hr / 2
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    x = (u - cx_r) / f
    y = (v - cy_r) / f
    z = 1
    ray_rect = np.array([x, y, z])
    ray_rect /= np.linalg.norm(ray_rect)
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_fish = Rot.T @ ray_rect
    Xf, Yf, Zf = ray_fish
    theta = np.arccos(np.clip(Zf, -1, 1))
    phi = np.arctan2(Yf, Xf)
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    r = (theta / theta_max) * radius
    u_f = cx_f + r * np.cos(phi)
    v_f = cy_f - r * np.sin(phi)
    return (int(u_f), int(v_f))

def fisheye_to_rect_point(u_f, v_f, view_params, fisheye_params):
    cx_f, cy_f = fisheye_params["cx"], fisheye_params["cy"]
    radius, theta_max = fisheye_params["radius"], fisheye_params["theta_max_rad"]

    dx, dy = u_f - cx_f, cy_f - v_f
    r = np.sqrt(dx ** 2 + dy ** 2)
    theta = (r / radius) * theta_max
    phi = np.arctan2(dy, dx)
    sin_t = np.sin(theta)
    ray_fish = np.array([sin_t * np.cos(phi), 
                         sin_t * np.sin(phi), 
                         np.cos(theta)])
    Rot = compute_basis(view_params["yaw"], view_params["pitch"])
    ray_rect = Rot @ ray_fish

    Xc, Yc, Zc = ray_rect
    if Zc <= 0:
        return None

    Hr, Wr = rect_h, rect_w
    fov = view_params["fov"]
    f = (Wr / 2) / np.tan(np.deg2rad(fov / 2))
    cx_r, cy_r = Wr / 2, Hr / 2
    u = f * (Xc / Zc) + cx_r
    v = f * (Yc / Zc) + cy_r
    if 0 <= u < Wr and 0 <= v < Hr:
        return (int(u), int(v))
    return None

def make_rect_grid_disp():
    grid = np.ones((RECT_TILE_H_DISP * 2, RECT_TILE_W_DISP * 2, 3), dtype=np.uint8) * 30
    grid[0:RECT_TILE_H_DISP, 0:RECT_TILE_W_DISP] = views_disp[0]
    grid[0:RECT_TILE_H_DISP, RECT_TILE_W_DISP:] = views_disp[1]
    grid[RECT_TILE_H_DISP:, 0:RECT_TILE_W_DISP] = views_disp[2]
    grid[RECT_TILE_H_DISP:, RECT_TILE_W_DISP:] = views_disp[3]
    return grid
dis_rect_w, dis_rect_h = 2592,2592
fov = 160.0
fov_RAD = np.deg2rad(fov)
focal = (dis_rect_w / 2) / np.tan(fov_RAD / 2)

dis_K_rect = np.array([[focal, 0.0, dis_rect_w / 2],
                   [0.0, focal, dis_rect_h / 2],
                   [0.0, 0.0, 1.0]])

f_fish = fisheye_params["radius"] / fisheye_params["theta_max_rad"]
K_fish = np.array([[f_fish, 0.0, fisheye_params["cx"]],
                   [0.0, f_fish, fisheye_params["cy"]],
                   [0.0, 0.0, 1.0]])
D_fish = np.zeros((4, 1)) 




scale_BEV=0.5
scale_rect = 0.5
FISH_W_DISP, FISH_H_DISP = 1280, 720
RECT_TILE_W_DISP, RECT_TILE_H_DISP = int(rect_w * scale_rect), int(rect_h * scale_rect)
bev_display = cv2.resize(BEV, (0,0), fx=scale_BEV, fy=scale_BEV)
views_disp = [cv2.resize(v, (0,0), fx=scale_rect, fy=scale_rect) for v in views]
fisheye_display = cv2.resize(fisheye_img, (FISH_W_DISP, FISH_H_DISP))

scale_fx = fisheye_params["width"] / FISH_W_DISP
scale_fy = fisheye_params["height"] / FISH_H_DISP
rect_grid_disp = make_rect_grid_disp()

def draw_text(img, text, pos, color=(255, 255, 255)):
    font = cv2.FONT_HERSHEY_COMPLEX  
    scale = 0.8                 
    thickness = 2         
    x, y = int(pos[0]), int(pos[1])
    cv2.putText(img, text, (x + 1, y + 1), font, scale, (0, 0, 0), thickness , cv2.LINE_AA)
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

def on_mouse(event, x, y, flags, param):
    global canvas
    if event != cv2.EVENT_LBUTTONDOWN:
        return

    fisheye_disp = fisheye_display.copy()
    rect_disp = rect_grid_disp.copy()
    bev_disp = bev_display.copy()

    if x < FISH_W_DISP:  
        u_full, v_full = x * scale_fx, y * scale_fy
        cv2.circle(fisheye_disp, (x, y), 3, (0, 255, 0), -1)
        draw_text(fisheye_disp, f"({u_full:.1f},{v_full:.1f})", (x, y), (0, 255, 0))
        print(f"Click Fisheye Point = ({u_full:.1f},{v_full:.1f})")

        for i, cam in enumerate(cam_settings):
            pt = fisheye_to_rect_point(u_full, v_full, cam, fisheye_params)
            if pt is not None:
                u_rect, v_rect = pt
                ox = (i % 2) * RECT_TILE_W_DISP
                oy = (i // 2) * RECT_TILE_H_DISP
                u_disp, v_disp = int(u_rect * scale_rect), int(v_rect * scale_rect)
                pos = (ox + u_disp, oy + v_disp)
                cv2.circle(rect_disp, pos, 3, (0, 0, 255), -1)
                draw_text(rect_disp, f"({u_rect},{v_rect})", pos, (0, 0, 255))
                print(f" Rect view {i+1}: Rect Point = ({u_rect:.1f},{v_rect:.1f}) ")

    else: 
        xr = x - FISH_W_DISP
        xi = xr % RECT_TILE_W_DISP
        yi = y % RECT_TILE_H_DISP
        view_idx = (y >= RECT_TILE_H_DISP) * 2 + (xr >= RECT_TILE_W_DISP)
        u_full, v_full = xi / scale_rect, yi / scale_rect
        print(f"Click Rect_point = ({u_full},{v_full}) ")
        draw_text(rect_disp, f"({u_full},{v_full})", (x - FISH_W_DISP, y), (0, 255, 0))
        cv2.circle(rect_disp, (x - FISH_W_DISP, y), 3, (0, 255, 0), -1)
        pt_f = rect_to_fisheye_point(u_full, v_full, cam_settings[view_idx], fisheye_params)
        if pt_f is not None:
            u_f, v_f = pt_f
            u_fd, v_fd = int(u_f / scale_fx), int(v_f / scale_fy)
            cv2.circle(fisheye_disp, (u_fd, v_fd), 3, (0, 0,255), -1)
            draw_text(fisheye_disp, f"({int(u_f)},{int(v_f)})", (u_fd, v_fd), (0, 0, 255))
            pt_f = np.array(pt_f)
            pt_f = pt_f.astype(np.float32).reshape(-1,1,2)
            rect_point = cv2.fisheye.undistortPoints(pt_f, K_fish, D_fish, P=dis_K_rect)
            mapped_pt = cv2.perspectiveTransform(rect_point, h)
            mx, my = int(mapped_pt[0][0][0]), int(mapped_pt[0][0][1])
            x_disp, y_disp = int(mx * scale_BEV), int(my * scale_BEV)
            cv2.circle(bev_disp, (x_disp, y_disp), 3, (0, 0, 255), -1)
            cv2.putText(bev_disp, f"({mx}, {my})", (x_disp + 10, y_disp - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            print(f" Fisheye Point = ({int(u_f)},{int(v_f)}) | BEV Point = ({mx},{my})")

            

    canvas = np.ones((max(FISH_H_DISP, rect_disp.shape[0]), FISH_W_DISP + rect_disp.shape[1], 3), dtype=np.uint8) * 255
    canvas[0:FISH_H_DISP, 0:FISH_W_DISP] = fisheye_disp
    canvas[0:rect_disp.shape[0], FISH_W_DISP:] = rect_disp
    cv2.imshow("BEV Image", bev_disp)
    cv2.imshow("Unified Mapping", canvas)

canvas = np.ones((rect_h, 2560, 3), dtype=np.uint8) * 255
canvas[0:rect_h, 0:rect_w] = fisheye_display
canvas[0:rect_h, rect_w:] = rect_grid_disp

cv2.namedWindow("Unified Mapping", cv2.WINDOW_NORMAL)
cv2.setMouseCallback("Unified Mapping", on_mouse)
cv2.imshow("Unified Mapping", canvas)
cv2.imshow("BEV Image", bev_display)

while True:
    if cv2.waitKey(20) & 0xFF == 27:
        break

cv2.destroyAllWindows()


Click Fisheye Point = (1425.6,1144.8)
Click Rect_point = (624.0,574.0) 
 Fisheye Point = (1698,1086) | BEV Point = (1292,665)
Click Rect_point = (692.0,490.0) 
 Fisheye Point = (1746,1175) | BEV Point = (1298,648)
Click Rect_point = (604.0,316.0) 
 Fisheye Point = (1931,1134) | BEV Point = (1279,594)
Click Rect_point = (732.0,224.0) 
 Fisheye Point = (1838,528) | BEV Point = (1186,658)
Click Rect_point = (676.0,406.0) 
 Fisheye Point = (1677,605) | BEV Point = (1242,693)
Click Rect_point = (732.0,526.0) 
 Fisheye Point = (1641,724) | BEV Point = (1262,692)
Click Rect_point = (456.0,452.0) 
 Fisheye Point = (1033,1018) | BEV Point = (1311,735)
Click Rect_point = (420.0,470.0) 
 Fisheye Point = (1034,1053) | BEV Point = (1314,733)
Click Rect_point = (988.0,350.0) 
 Fisheye Point = (1198,1631) | BEV Point = (1381,674)
Click Rect_point = (968.0,180.0) 
 Fisheye Point = (1251,1769) | BEV Point = (1451,621)
Click Rect_point = (692.0,286.0) 
 Fisheye Point = (1517,1619) | BEV Point = (1370,62